In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient 
import dotenv
import requests
from bs4 import BeautifulSoup
import re
import datetime


In [2]:
oai_api_key = dotenv.get_key('.env.local', 'OPENAI_API_KEY')

In [3]:
async def get_abasin_info() -> str:
    f"""Get raw conditions data from Arapahoe Basin's website."""
    response = requests.get('https://www.arapahoebasin.com/snow-report/')
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        condition_divs = soup.find_all('div', class_=['conditions', 'weather'])
        return condition_divs
    return []

In [ ]:
results = await get_abasin_info()
print(results)

In [6]:
async def main() -> None:
    current_date = datetime.datetime.now().strftime("%Y-%m-%d")
    abasin_agent = AssistantAgent(
        name="abasin_agent",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key=oai_api_key,
        ),
        tools=[get_abasin_info],
        system_message=f"Return a short conditions assessment followed by a markdown summary of A Basin's conditions. The current date is {current_date}.",
    )

    ski_agent = AssistantAgent(
        name="ski_agent",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key=oai_api_key,
        ),
        tools=[],
        system_message=f"Describe why or why the user should not go skiing at a given time. The current date is {current_date}.",
    )

    termination = TextMentionTermination("TERMINATE")
    agent_team = RoundRobinGroupChat([ski_agent, abasin_agent], termination_condition=termination)

    stream = agent_team.run_stream(task="Should I go ski tomorrow?")
    await Console(stream)

In [ ]:
await main()